In [1]:
import requests
from bs4 import BeautifulSoup
from openpyxl import load_workbook

url = 'https://trevi.ctld.ncku.edu.tw/site/trevi_list'
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.parser")

In [2]:
class course():
    def __init__(self, title = None, content = None):
        self.title = title
        self.content = content
        self.category_num = []
        self.category_ch = []
        self.title_cut = []
        self.content_cut = []
        
    def print_info(self):
        print('title: ', self.title)
        print('title_cut: ', self.title_cut)
        print('content: ', self.content)
        print('content_cut: ', self.content_cut)
        print('category: ', self.category_ch)
        print('=========================================\n')

In [3]:
num_dict = "0123456789"

In [4]:
course_title = soup.find_all(class_="wish-course")
course_content = soup.find_all(class_="wish-txt")

In [5]:
courses = []

for i in course_title:
    courses.append(course(i.text))
    
for i, j in enumerate(course_content):
    try:
        courses[i].content = (j.text)
    except:
        pass

In [6]:
course_category = soup.find_all(class_="type ulset")
for i, j in enumerate(course_category):
    c = []
    for q in str(j):
        if q in num_dict:
            c.append(q)
    courses[i].category_num = c[0:-1]

In [7]:
for i in courses:
    ch = []
    for j in i.category_num:
        if j == '1':
            ch.append('生活時事')
        elif j == '2':
            ch.append('人文藝術')
        elif j == '3':
            ch.append('體能運動')
        elif j == '4':
            ch.append('設計手作')
        elif j == '5':
            ch.append('影音創作')
        elif j == '6':
            ch.append('科技程式')
        elif j == '7':
            ch.append('商業經營')
        elif j == '8':
            ch.append('國際語言')
        elif j == '9':
            ch.append('其他')
    i.category_ch = ch

In [8]:
wb = load_workbook('./crawler/crawler_result_forchuk_label.xlsx')
sheet = wb["sheet"]

In [9]:
r = 2

for i in courses:
    sheet.cell(row = r, column = 1, value = i.title)
    sheet.cell(row = r, column = 11, value = i.content)
    for j in range(2, 11):
        sheet.cell(row = r, column = j, value = 0)
    for j in i.category_num:
        sheet.cell(row = r, column = int(j)+1, value = 1)
    r = r + 1
    
wb.save('./crawler/crawler_result_forchuk_label.xlsx')

================================================================

In [105]:
import jieba

stopwords_dict_1 = [line.strip() for line in open('./jieba_dict/stopword_zh_tw.txt', 'r', encoding='utf-8').readlines()]
stopwords_dict_2 = [line.strip() for line in open('./jieba_dict/goto456_stopword_zh_tw.txt', 'r', encoding='utf-8').readlines()]

stopwords_dict_1.append('(')
stopwords_dict_1.append(')')

In [106]:
def cut_and_join(file):
    data_cut = []
    file_join = ''

#     seg_list_standard = jieba.lcut(file) #精確模式
    seg_list_standard = jieba.lcut_for_search(file) #搜尋引擎模式
    
    #remove stopwords
    for k, i in enumerate(seg_list_standard):
        if i in stopwords_dict_1:
            seg_list_standard.remove(i)
    
    for k, i in enumerate(seg_list_standard):
        if i in stopwords_dict_2:
            seg_list_standard.remove(i)

    #remove null data
    for k, i in enumerate(seg_list_standard):
        if i == ' ':
            seg_list_standard.remove(i)

    for i in seg_list_standard:
        file_join += i
        file_join += ' '

    return file_join

In [107]:
for i in courses:
    i.title = i.title.split('我想學')[1][:-1]
    i.title_cut = cut_and_join(i.title)
    i.content_cut = cut_and_join(i.content)

In [108]:
for i in courses:
    i.print_info()

title:  由簡入深notion使用技巧 
title_cut:  簡入 深 notion 使用 技巧 
content:  Notion筆記軟體是一套非常靈活的好用工具，整合了線上筆記本、文件管理、專案管理、知識資料庫等功能，為一款整合多功能的實用工具。從入門到精通Notion，希望透過學會使用該工具建立適合自己的系統，並且將其運用在生活與工作上！
content_cut:  Notion 筆記 軟體 一套 靈活 工具 整合 線 筆記本 文件 管理 專案 管理 知識 資料 庫 功能 一款 整合 功能 多功能 實用 工具 從入 門到 精通 Notion 希望 透過 學會 使用 工具 建立 適合 系統 將其 運用 生活 工作 
category:  ['生活時事', '設計手作']

title:  教你把錢管好ㄉ⇝人生必學的理財八堂課 ö
title_cut:  教 錢 管好 ㄉ ⇝ 人生 必學 理 財八堂 課 ö 
content:  花錢如流水～一直賺一直花！一直賺一直花！
爸媽總說多賺點錢，哎呀，還不如來學點理財啦
（可以的話希望邀請李柏鋒（鋒哥）來上課）
content_cut:  花錢 流水 一直 賺 一直 花 一直 賺 一直 花 
 爸媽 總 說 賺點 錢 還 來學點 理財 
 話 希望 邀請 李柏鋒 鋒哥 課 
category:  ['生活時事', '商業經營']

title:  薩提爾的對話練習
title_cut:  薩提爾 話 練習 
content:  「以好奇的姿態，理解你的內在冰山，探索自己，連結他人。」
希望可以透過學習薩提爾與了解冰山模式，回歸到最初對於「人」的關注，
無關於對或錯好或壞，而是期望能夠察覺自身以及他人情緒背後的動機或渴望，
並進一步展開對話，療癒他人也療癒自我。

content_cut:  好奇 姿態 理解 冰山 探索 連結 
 希望 透過 學習薩 提爾 了解 冰山 模式 回歸到 最初 的 關注 
 無關 錯 壞 期望 能夠 察覺 人情 緒 背 的 動機 渴望 
 並進 一步 展開 話 療 癒 也療 癒 自我 
 
category:  ['人文藝術']

title:  TouchDesigner多媒體互動設計(實務應用)
title_cut:  TouchDesigner 多媒體 互動 設計 實務 應用 


In [124]:
wb = load_workbook('./crawler/crawler_result_forchuk_label.xlsx')
sheet = wb["sheet"]

r = 2

for i in courses:
    all_list = i.title_cut.split(' ') + i.content_cut.split(' ')
    for j in all_list:
        if j != None and j != '' and j != '\n' and j != '\t' and len(j) >= 2:
            sheet.cell(row = r, column = 11, value = '')
            sheet.cell(row = r, column = 1, value = j)
            for q in range(2, 11):
                sheet.cell(row = r, column = q, value = 0)
            for q in i.category_num:
                sheet.cell(row = r, column = int(q)+1, value = 1)
            r = r + 1

    
wb.save('./crawler/crawler_result_forchuk_label.xlsx')